In [ ]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import os
from datetime import datetime
from dateutil.parser import parse
from waterallocationsFunctions import *

In [ ]:
# working directory
working_dir = "C:/tseg/NMTest/aggregatedamounts/"
os.chdir(working_dir)

In [ ]:
target_columns = ["ReportingUnitUUID", "ReportingUnitNativeID", "ReportingUnitName", 
                  "ReportingUnitTypeCV", "ReportingUnitUpdateDate", "ReportingUnitProductVersion",
                  "StateCV", "EPSGCodeCV", "Geometry"]

In [ ]:
outdf100 = pd.DataFrame(columns=target_columns)

In [ ]:
# Input files
fileInput1 = "Summary of withdrawals by county 90-15.xlsx" 
fileInput2 = "Summary of withdrawals by River Basin 90-15.xlsx" 

In [ ]:
df10 = pd.read_excel(fileInput1, header=0, sheet_name=0, skiprows=1, encoding = "ISO-8859-1")
#df20 = pd.read_excel(fileInput2, header=0, sheet_name=0, skiprows=1, encoding = "ISO-8859-1")
list(df10.columns)
#list(df20.columns)

In [ ]:
# combine multiple sheets to one dataFrame

startYear = 1990
endYear = 2015
numSheets = 5
yearList = np.linspace(startYear, endYear, numSheets)
df100_list = []
for isx in range (numSheets):
    df10 = pd.read_excel(fileInput1, header=0, sheet_name=isx, skiprows=1, encoding = "ISO-8859-1")
    df10 = df10.assign(ReportYearCV=yearList[isx])
    df10.ReportYearCV = df10.ReportYearCV.astype(int)
    df100_list.append(df10)
    
df100 = pd.concat(df100_list, sort=True)

df100

In [ ]:
print("Copying County name...")

# all we need from input is county name

# first remove redundancies
df100 = df100.drop_duplicates(subset=["COUNTY"])   #
df100 = df100.reset_index(drop=True)

print(len(df100.index))

outdf100["ReportingUnitName"] = df100["COUNTY"]

In [ ]:
# ReportingUnitNativeID	 	 Auto generate
outdf100['ReportingUnitNativeID'] = range(1, len(outdf100.index) + 1)

#ReportingUnitUUID	 	 NM_NativeID
outdf100['ReportingUnitUUID'] = outdf100.apply(lambda row: 
                                              "_".join(["NM", str(row['ReportingUnitNativeID'])]),
                                                axis=1)


In [ ]:
import shapefile
import pygeoif
import os
import geopandas as gpd

print("Geometry...")

working_dir = "C:/tseg/MappingStatesDataToWaDE2.0/NewMexico/WaterAllocation/NMShapefile/"
os.chdir(working_dir)

#Geometry	Get it from shapefile as WKT
shapefilName = 'OSE_Points_of_Diversion.shp'
nm_counties = "NM_Historical_Counties.shp"
us_counties = "US_County_Boundaries.shp"
basins = "WUR_surface_Basins.shp"


sf = shapefile.Reader(basins)

print(sf)

shapes = sf.shapes()

shapes[0].shapeType

fields = sf.fields

print (fields)

gm= pygeoif.geometry.as_shape(shapes[0])
#print (gm.wkt)

#print (fields[0])
records = sf.records()
print(records)


In [2]:
import json
import shapefile
import pygeoif
import os
import geopandas as gpd

working_dir = "C:/tseg/MappingStatesDataToWaDE2.0/NewMexico/WaterAllocation/NMShapefile/"
os.chdir(working_dir)

#Geometry	Get it from shapefile as WKT
shapefilName = 'OSE_Points_of_Diversion.shp'
nm_counties = "NM_Historical_Counties.shp"
us_counties = "US_County_Boundaries.shp"
basins = "WUR_surface_Basins.shp"

geojson_file = "basins_geojson"
geodf = gpd.read_file(basins)
geodf.to_file(geojson_file, driver = "GeoJSON")

with open(geojson_file) as geofile:
    basinsJson = json.load(geofile)

"""
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    basinsJson = json.load(response)

i=1
for feature in basinsJson["features"]:
    #feature ['id'] = str(i).zfill(2)
    #locations.append(str(i).zfill(2))
    zvals.append(1.0*i/6.0)
    i += 1
"""
"""
locations =[]
zvals = []

if 'id' not in basinsJson['features'][0].keys():
    if 'properties' in basinsJson['features'][0].keys():
        if 'id' in basinsJson['features'][0]['properties'] and basinsJson['features'][0]['properties']['id'] is not None:
            for k, feat in enumerate(basinsJson['features']):
                basinsJson['features'][k]['id'] = feat['properties']['id']
        else:
            for k in range(len(basinsJson['features'])):
                basinsJson['features'][k]['id'] = str(k+1).zfill(2)
                basinsJson['features'][k]['properties']['id'] = str(k+1).zfill(2)
"""
for k in range(len(basinsJson['features'])):
                basinsJson['features'][k]['id'] = str(k+1).zfill(2)
                basinsJson['features'][k]['properties']['id'] = str(k+1).zfill(2)
            
for k in range(len(basinsJson['features'])):
    locations.append(basinsJson['features'][k]['id'])  
    zvals.append(100.0*k)

print(len(locations))
print(len(zvals))
print(len(basinsJson['features']))
#for k in range(len(basinsJson['features'])):basinsJson['features'][k]['id'] = str(k+1).zfill(2)
        
#basinsJson
#print(locations)
idS = [basinsJson['features'][k]['id'] for k in range(len(basinsJson['features']))]
print(locations)
print(idS)
print(zvals)
#print(basinsJson['features'][5].keys())

#print(basinsJson['features'][0]['properties'])
#print(basinsJson['features'][0])

In [ ]:
text=[feat['properties']['REG_NAME'] for feat in basinsJson['features'] if feat['id'] in locations]
text

In [ ]:
token = open("../.mapbox_token").read() 

import plotly.graph_objects as go
import numpy as np
   
trace = go.Choroplethmapbox(z=zvals,
                            locations=locations,
                            colorscale="Viridis",
                            colorbar=dict(thickness=20, ticklen=3),
                            geojson=basinsJson,
                            text=text,
                            hoverinfo='all',
                            marker_line_width=0.1, marker_opacity=0.7)
                            
                            
layout = go.Layout(title_text= 'Choroplethmapbox',
                   title_x=0.5, width = 700, height=700,
                   mapbox = dict(center= dict(lat=35.1,  lon=-106.6),
                                 accesstoken= token,
                                 zoom=3,
                               ))

fig = go.Figure(data=[trace], layout =layout)
fig.update_layout(mapbox_style = "light")
fig.show()

In [ ]:
token = open("../.mapbox_token").read() 

import plotly.graph_objects as go
import numpy as np



fig = go.Figure(go.Choroplethmapbox(geojson=basinsJson, locations=locations, z=zvals,
                                    colorscale="Viridis", zmin=0, zmax=6.0, marker_line_width=0.5, marker_opacity=0.7))
fig.update_layout(mapbox_style="light", mapbox_accesstoken=token,
                  mapbox_zoom=5, mapbox_center = {"lat": 35.1, "lon": -106.6})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:

"""
shapeRecs = sf.shapeRecords()

for shaperec in r.iterShapeRecords():
    w.record(*shaperec.record)


fields = sf.fields[1:] 
field_names = [field[0] for field in fields] 
# construction of a dctionary field_name:value  
for r in sf.shapeRecords():  
    atr = dict(zip(field_names, r.record)) 
    if atr['STATE'] == 'New Mexico':
        action

for shape in sf.iterShapes():
...     # do something here
...     pass

>>> for rec in sf.iterRecords():
...     # do something here
...     pass

>>> for shapeRec in sf.iterShapeRecords():
...     # do something here
...     pass

>>> for shapeRec in sf: # same as iterShapeRecords()
...     # do something here
...     pass

gm=[]

for sp in shapes:
    gm.append(pygeoif.geometry.as_shape(sp)) 

m = pygeoif.MultiPoint(gm)

print (m.wkt)
"""

In [ ]:
# hardcoded

outdf100.StateCV = "NM"
outdf100.ReportingUnitTypeCV = "County"
outdf100.EPSGCodeCV = "EPSG:4326"
sampleWKT =  'POLYGON((-99.54319297853704 37.15853229006052, -97.26976797641987 37.15759429005948, -105.11636298372741 37.14764529005038, -104.52740598317905 37.15119229005359, -104.09963198278069 37.15376929005606, -103.56062798227867 37.156443290058405, -103.12301898187116 37.157137290059154, -103.08639398183686 37.15689329005886, -103.00203898175846 37.156332290058344, -99.90287697887197 37.162385290064094, -99.54319297853704 37.15853229006052))'
outdf100.Geometry = sampleWKT

# replace NaN with blank cells
outdf100 = outdf100.replace(np.nan, '')
outdf100.head(5)

In [ ]:
print("Writing out...")

#write out
out_repunit = 'reportingunits.csv'
outdf100.to_csv(out_repunit, index=False, encoding = "utf-8")

print("Done sites")